# FindCare — Gradio Backend (Notebook)

**File:** FindCare_Gradio_Backend.ipynb  
**Author:** Skip Snow  
**Co-Author:** GPT-5  
**Copyright:** Copyright (c) 2025 Skip Snow. All rights reserved.

This notebook implements a **Gradio + FastAPI** backend that matches the frontend webhook/API expectations in your provided spec.


In [ ]:
# If needed, install dependencies (uncomment in a fresh environment)
# !pip -q install gradio fastapi uvicorn python-multipart pydantic

import os
import gradio as gr
from fastapi import Request, UploadFile, File, Form
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional, Dict, Any
import uuid
import time
import json
import re
import socket
import socket
import socket


## 1) In-memory session store (replace with Redis/Mongo later)

The frontend currently sends **no session id**, so we keep a simple global session. If you later add a header or cookie, you can key sessions by that value.

In [11]:
# In-memory session storage (keyed by session ID from cookie)
# In production, replace with Redis/MongoDB
_sessions: Dict[str, dict] = {}

def get_or_create_session_id(request: Request) -> str:
    """Get session ID from cookie, or create a new one."""
    session_id = request.cookies.get("findCareSession")
    if not session_id:
        session_id = str(uuid.uuid4())
    return session_id

def get_session(session_id: str) -> dict:
    """Get session data, creating if it doesn't exist."""
    if session_id not in _sessions:
        _sessions[session_id] = {
            "created_at": time.time(),
            "prompts": [],
            "results": [],
            "copies": [],
            "deletes": [],
            "selected_state": None,
            "model": "mock-default",
        }
    return _sessions[session_id]

def set_session_cookie(response: JSONResponse, session_id: str):
    """Set the secure cookie with session ID."""
    # Use secure=True in production (HTTPS), False for local development (HTTP)
    is_production = os.getenv("ENVIRONMENT", "development").lower() == "production"
    
    response.set_cookie(
        key="findCareSession",
        value=session_id,
        httponly=True,
        secure=is_production,  # Only send over HTTPS in production
        samesite="lax",
        max_age=86400 * 30  # 30 days
    )

class SessionStore:
    """Wrapper to access session data for a given request."""
    
    def __init__(self, request: Request, response: Optional[JSONResponse] = None):
        self.session_id = get_or_create_session_id(request)
        self._session = get_session(self.session_id)
        self.response = response
        
    def add_prompt(self, prompt: str):
        self._session["prompts"].append(prompt)
        
    def set_results(self, results: List[dict]):
        self._session["results"] = results
        
    def delete_result(self, result_id: int):
        self._session["deletes"].append({"resultId": result_id, "ts": time.time()})
        self._session["results"] = [r for r in self._session["results"] if r.get("id") != result_id]
        
    def copy_result(self, result_id: int):
        self._session["copies"].append({"resultId": result_id, "ts": time.time()})
        
    def set_state(self, state: str):
        self._session["selected_state"] = state
        
    def set_model(self, model_id: str):
        self._session["model"] = model_id
        
    def transcript_text(self) -> str:
        lines = []
        for i, p in enumerate(self._session["prompts"], start=1):
            lines.append(f"User[{i}]: {p}")
        return "\n".join(lines)
        
    def summary_text(self) -> str:
        if not self._session["prompts"]:
            return "No queries yet."
        last = self._session["prompts"][-1]
        return f"Session includes {len(self._session['prompts'])} query(ies). Most recent request: {last}"
        
    @property
    def data(self):
        return self._session
        
    def set_cookie_on_response(self, response: JSONResponse):
        """Set the session cookie on the response."""
        set_session_cookie(response, self.session_id)


## 2) Mock provider search

Replace this with your real provider/specialty lookup (Mongo, vector search, etc.).

In [12]:
STATE_ABBR = {
    "Alabama":"AL","Alaska":"AK","Arizona":"AZ","Arkansas":"AR","California":"CA","Colorado":"CO","Connecticut":"CT",
    "Delaware":"DE","Florida":"FL","Georgia":"GA","Hawaii":"HI","Idaho":"ID","Illinois":"IL","Indiana":"IN","Iowa":"IA",
    "Kansas":"KS","Kentucky":"KY","Louisiana":"LA","Maine":"ME","Maryland":"MD","Massachusetts":"MA","Michigan":"MI",
    "Minnesota":"MN","Mississippi":"MS","Missouri":"MO","Montana":"MT","Nebraska":"NE","Nevada":"NV","New Hampshire":"NH",
    "New Jersey":"NJ","New York":"NY","North Carolina":"NC","North Dakota":"ND","Ohio":"OH","Oklahoma":"OK","Oregon":"OR",
    "Pennsylvania":"PA","Rhode Island":"RI","South Carolina":"SC","South Dakota":"SD","Tennessee":"TN","Texas":"TX","Utah":"UT",
    "Vermont":"VT","Virginia":"VA","Washington":"WA","West Virginia":"WV","Wisconsin":"WI","Wyoming":"WY"
}

TYPE_COLOR = {
    "Hospital": "#3b82f6",
    "Clinic": "#10b981",
    "Urgent Care": "#f59e0b",
    "Private Practice": "#8b5cf6",
    "Imaging Center": "#ef4444",
}

MOCK_PROVIDERS = [
    {"name":"General Hospital", "type":"Hospital", "states":["CA","TX","FL","NY"], "distance":"2.3 miles"},
    {"name":"Family Care Clinic", "type":"Clinic", "states":["CA","WA","OR","NV"], "distance":"1.8 miles"},
    {"name":"Urgent Care Center", "type":"Urgent Care", "states":["TX","LA","OK","AR"], "distance":"3.5 miles"},
    {"name":"Heart & Vascular Associates", "type":"Private Practice", "states":["CA","NY","PA","MA"], "distance":"4.1 miles"},
    {"name":"Advanced Imaging", "type":"Imaging Center", "states":["FL","GA","NC","SC"], "distance":"2.9 miles"},
]

def _infer_state_from_text(prompt: str) -> Optional[str]:
    # Accept "CA" or full state name
    m = re.search(r"\b([A-Z]{2})\b", prompt)
    if m:
        abbr = m.group(1)
        if abbr in set(STATE_ABBR.values()):
            return abbr

    lowered = prompt.lower()
    for name, abbr in STATE_ABBR.items():
        if name.lower() in lowered:
            return abbr
    return None

def mock_search(prompt: str, selected_state: Optional[str]=None) -> List[dict]:
    # Very simple intent heuristics
    state = selected_state or _infer_state_from_text(prompt)
    needle = prompt.lower().strip()

    results = []
    base_id = int(time.time() * 1000)

    for i, p in enumerate(MOCK_PROVIDERS):
        # Filter by state if available
        if state and state not in p["states"]:
            continue

        # Very light text filter by keywords
        if needle:
            if ("cardio" in needle) and ("Heart" not in p["name"]):
                # keep only the heart practice for cardio queries
                continue
            if ("urgent" in needle) and (p["type"] != "Urgent Care"):
                continue
            if ("imaging" in needle or "xray" in needle or "mri" in needle) and (p["type"] != "Imaging Center"):
                continue

        results.append({
            "id": base_id + i,
            "name": p["name"],
            "type": p["type"],
            "color": TYPE_COLOR.get(p["type"], "#6b7280"),
            "distance": p["distance"],
        })

    # Always return something (frontend shows "No results yet" otherwise)
    if not results:
        results = [{
            "id": base_id,
            "name": "No matching providers (mock)",
            "type": "Clinic",
            "color": TYPE_COLOR["Clinic"],
            "distance": "N/A",
        }]
    return results


## 3) File handling stub

The frontend sends uploaded files as `multipart/form-data`. Here we extract lightweight text (filename + size). Replace with PDF/DOCX parsing + OCR when you wire it up.

In [13]:
async def summarize_uploads(files: Optional[List[UploadFile]]) -> str:
    if not files:
        return ""
    parts = []
    for f in files:
        try:
            raw = await f.read()
            parts.append(f"{f.filename} ({len(raw)} bytes)")
        except Exception:
            parts.append(f"{getattr(f,'filename','(unknown)')} (unreadable)")
    return " | ".join(parts)


## 4) FastAPI models for JSON endpoints

In [14]:
class ResultIdPayload(BaseModel):
    resultId: int

class CopyPayload(BaseModel):
    resultId: int

class ActionPayload(BaseModel):
    action: str  # insurance, emr, transcript, models

class StatePayload(BaseModel):
    state: str  # can be 'California' or 'CA'
    action: Optional[str] = "select"

class LinkPayload(BaseModel):
    resultId: int

class NavigationPayload(BaseModel):
    page: str


## 5) Gradio app + API routes

We expose all endpoints your React code calls:
- `POST /api/search` (multipart)
- `POST /api/query` (multipart; same behavior)
- `DELETE /api/result`
- `POST /api/copy`
- `POST /api/action`
- `POST /api/state`
- `POST /api/link`
- `POST /api/navigation` (in spec; safe to have)

Also adds CORS so your React app can call the backend.

In [15]:
def build_gradio_ui():
    # For Gradio UI testing, create a mock session store
    # (Gradio UI doesn't have request context like API endpoints do)
    mock_session = {
        "created_at": time.time(),
        "prompts": [],
        "results": [],
        "copies": [],
        "deletes": [],
        "selected_state": None,
        "model": "mock-default",
    }
    
    class MockStore:
        def __init__(self):
            self._session = mock_session
        def add_prompt(self, p): self._session["prompts"].append(p)
        def set_results(self, r): self._session["results"] = r
        @property
        def data(self): return self._session
    
    mock_store = MockStore()
    
    with gr.Blocks(title="FindCare Backend (Dev)") as demo:
        gr.Markdown("## FindCare Backend (Dev Panel)\nUse this for quick backend testing while your React UI is wiring up.")
        with gr.Row():
            prompt = gr.Textbox(label="Prompt", lines=3, placeholder="Find cardiologists in CA who accept Blue Cross")
            files = gr.File(label="Files", file_count="multiple")
        out = gr.JSON(label="Search response")
        btn = gr.Button("Run mock /api/search")

        def _run(prompt_text, uploaded_files):
            mock_store.add_prompt(prompt_text or "")
            results = mock_search(prompt_text or "", mock_store.data.get("selected_state"))
            mock_store.set_results(results)
            return {"results": results}

        btn.click(_run, inputs=[prompt, files], outputs=[out])

        gr.Markdown("### Session snapshot")
        sess = gr.JSON()
        refresh = gr.Button("Refresh session store")
        refresh.click(lambda: mock_store.data, inputs=[], outputs=[sess])
    return demo

demo = build_gradio_ui()
app = demo.app  # FastAPI app


In [16]:
# CORS: allow your React dev server + local variants. Tighten for production.
app.add_middleware(
    CORSMiddleware,
    allow_origins=[
        "http://localhost:5173",
        "http://127.0.0.1:5173",
        "http://localhost:3000",
        "http://127.0.0.1:3000",
    ],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


In [17]:
def _normalize_state(state: str) -> str:
    state = (state or "").strip()
    if not state:
        return ""
    if len(state) == 2 and state.isalpha():
        return state.upper()
    # map full name to abbr
    for name, abbr in STATE_ABBR.items():
        if name.lower() == state.lower():
            return abbr
    return state[:2].upper()

@app.post("/api/search")
async def api_search(request: Request, prompt: str = Form(...), files: Optional[List[UploadFile]] = File(None)):
    store = SessionStore(request)
    store.add_prompt(prompt)
    file_summary = await summarize_uploads(files)
    _ = file_summary  # Can incorporate file_summary into real LLM prompt/context later

    results = mock_search(prompt, store.data.get("selected_state"))
    store.set_results(results)
    
    response = JSONResponse({"results": results})
    store.set_cookie_on_response(response)
    return response

@app.post("/api/query")
async def api_query(request: Request, prompt: str = Form(...), files: Optional[List[UploadFile]] = File(None)):
    # Same behavior as /api/search (frontend calls both)
    return await api_search(request=request, prompt=prompt, files=files)

@app.delete("/api/result")
async def api_delete_result(request: Request, payload: ResultIdPayload):
    store = SessionStore(request)
    store.delete_result(payload.resultId)
    response = JSONResponse({"ok": True})
    store.set_cookie_on_response(response)
    return response

@app.post("/api/copy")
async def api_copy(request: Request, payload: CopyPayload):
    store = SessionStore(request)
    store.copy_result(payload.resultId)
    response = JSONResponse({"ok": True})
    store.set_cookie_on_response(response)
    return response

@app.post("/api/action")
async def api_action(request: Request, payload: ActionPayload):
    store = SessionStore(request)
    action = (payload.action or "").strip().lower()

    if action == "insurance":
        response = JSONResponse({"ok": True, "url": "https://example.com/insurance-portal"})
        store.set_cookie_on_response(response)
        return response
    if action == "emr":
        response = JSONResponse({"ok": True, "url": "https://example.com/emr-access"})
        store.set_cookie_on_response(response)
        return response
    if action == "transcript":
        response = JSONResponse({
            "transcript": store.transcript_text(),
            "summary": store.summary_text(),
            "downloadUrl": None
        })
        store.set_cookie_on_response(response)
        return response
    if action == "models":
        available = [
            {"id": "mock-default", "name": "Mock Default", "description": "No-cost stub model."},
            {"id": "openai-gpt", "name": "OpenAI (future)", "description": "Placeholder for OpenAI integration."},
            {"id": "local-llama", "name": "Local Llama (future)", "description": "Placeholder for local model."},
        ]
        response = JSONResponse({"availableModels": available, "currentModel": store.data.get("model")})
        store.set_cookie_on_response(response)
        return response

    response = JSONResponse({"error": f"Unknown action: {payload.action}", "code": "BAD_ACTION"}, status_code=400)
    store.set_cookie_on_response(response)
    return response

@app.post("/api/state")
async def api_state(request: Request, payload: StatePayload):
    store = SessionStore(request)
    abbr = _normalize_state(payload.state)
    store.set_state(abbr)

    # Return a provider list filtered by state, plus minimal state info.
    providers = mock_search("", selected_state=abbr)
    store.set_results(providers)

    response = JSONResponse({
        "providers": providers,
        "stateInfo": {
            "name": abbr,
            "insuranceCoverage": "Mock coverage info (replace with real content).",
            "regulations": "Mock regulations info (replace with real content)."
        }
    })
    store.set_cookie_on_response(response)
    return response

@app.post("/api/link")
async def api_link(request: Request, payload: LinkPayload):
    store = SessionStore(request)
    rid = payload.resultId
    response = JSONResponse({"url": f"https://example.com/provider/{rid}"})
    store.set_cookie_on_response(response)
    return response

@app.post("/api/navigation")
async def api_navigation(request: Request, payload: NavigationPayload):
    store = SessionStore(request)
    page = (payload.page or "").strip().lower()
    response = JSONResponse({"ok": True, "page": page})
    store.set_cookie_on_response(response)
    return response


## 6) Launch

- In a notebook: use `prevent_thread_lock=True`.
- For React dev: point your frontend dev server proxy to this backend, or run both on localhost and call these endpoints directly.

In [18]:
# Launch Gradio (notebook-friendly)
# Try port 7860, if busy use next available port

def find_free_port(start_port=7860, max_attempts=10):
    """Find a free port starting from start_port."""
    for i in range(max_attempts):
        port = start_port + i
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('127.0.0.1', port))
                return port
            except OSError:
                continue
    return None  # Could not find free port

port = find_free_port(7860)
if port is None:
    # Let Gradio auto-select a port
    print("Could not find free port starting from 7860, letting Gradio auto-select...")
    demo.queue().launch(
        server_name="127.0.0.1",
        server_port=None,  # Auto-select port
        show_api=False,
        prevent_thread_lock=True
    )
else:
    print(f"Launching Gradio on port {port}...")
    demo.queue().launch(
        server_name="127.0.0.1",
        server_port=port,
        show_api=False,
        prevent_thread_lock=True
    )


NameError: name 'socket' is not defined